<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-RF" data-toc-modified-id="GS-for-RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GS for RF</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li></ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
nb_name = "svm_nb_lr_RF_baseline_v2_up-sampling_augmentation_stem_voc"

In [4]:
import numpy as np 
import pandas as pd
import random

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
setup_seed_ml(2021)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
train, test = load_data(only_stem_voc=True)
train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
# train_text, train_label = train_augmentation(train, select_comb=[['text']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']
combination 2 train:  ['reply', 'reference_one']


In [6]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label.values
y_test = test_label.values

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(23626,)
(7761,)
(23626,)
(7761,)


# GS for NB

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
parameters = {'tfidf__min_df': [1, 3], 'tfidf__stop_words': [ 'english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
              'clf__alpha': [0.2,0.4,0.6, 0.8, 1]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")
set(y_test)

clf__alpha:	0.2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.874


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=True, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 31635
X_train.shape (23626, 31635)
X_test.shape (7761, 31635)
classification_report:
f1:  {'micro avg': 0.7182064166988789, 'macro avg': 0.699287444034201} 

                          precision  recall  f1-score  support
alt.atheism                   0.715   0.511     0.596    319.0
comp.graphics                 0.661   0.612     0.636    389.0
comp.os.ms-windows.misc       0.620   0.538     0.576    394.0
comp.sys.ibm.pc.hardware      0.547   0.638     0.589    392.0
comp.sys.mac.hardware         0.694   0.584     0.635    385.0
comp.windows.x                0.683   0.676     0.679    395.0
misc.forsale                  0.841   0.731     0.782    390.0
rec.autos                     0.773   0.818     0.795    395.0
rec.motorcycles               0.841   0.874     0.857    398.0
rec.sport.baseball            0.679   0.897     0.773    397.0
rec.sport.hockey              0.972   0.757     0.851    827.0
sci.crypt                     0.776   0.884     0.826    396.0
sci.elect

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,0.067659,0.017283,0.877546,0.972282,0.872016,0.971387,0.872508,0.971652,0.872016,0.971387
1,0.061284,0.018943,0.867603,0.971319,0.864229,0.970145,0.863900,0.970492,0.864229,0.970145
2,0.058433,0.023458,0.894203,0.959148,0.889773,0.957901,0.890641,0.958217,0.889773,0.957901
3,0.058899,0.022058,0.886149,0.957719,0.882323,0.956490,0.882784,0.956784,0.882323,0.956490


# GS for SVM

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
parameters = {'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
             'clf__penalty':['l2'], 'clf__C':[1,2,3,4,5]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

clf__C:	2
clf__penalty:	l2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.907


## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=False, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 31962
X_train.shape (23626, 31962)
X_test.shape (7761, 31962)
classification_report:
f1:  {'micro avg': 0.7187218141992012, 'macro avg': 0.7032705236360972} 

                          precision  recall  f1-score  support
alt.atheism                   0.699   0.583     0.636    319.0
comp.graphics                 0.619   0.602     0.610    389.0
comp.os.ms-windows.misc       0.580   0.581     0.580    394.0
comp.sys.ibm.pc.hardware      0.547   0.620     0.581    392.0
comp.sys.mac.hardware         0.617   0.639     0.628    385.0
comp.windows.x                0.691   0.635     0.662    395.0
misc.forsale                  0.762   0.823     0.792    390.0
rec.autos                     0.768   0.787     0.778    395.0
rec.motorcycles               0.887   0.884     0.886    398.0
rec.sport.baseball            0.693   0.882     0.776    397.0
rec.sport.hockey              0.980   0.785     0.872    827.0
sci.crypt                     0.858   0.841     0.849    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,5.057593,0.020521,0.890444,0.997571,0.889961,0.997517,0.890137,0.997514,0.889961,0.997517
1,4.976637,0.019164,0.889539,0.997508,0.888776,0.997460,0.889173,0.997458,0.888776,0.997460
2,4.201133,0.025627,0.942858,0.997602,0.942431,0.997573,0.942388,0.997565,0.942431,0.997573
3,4.384151,0.023526,0.932299,0.998142,0.931256,0.998138,0.931221,0.998131,0.931256,0.998138


In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, dense=False, stop_words=True, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# cv

num of words: 73109
X_train.shape (23626, 73109)
X_test.shape (7761, 73109)
classification_report:
f1:  {'micro avg': 0.7188506635742816, 'macro avg': 0.7033165977467821} 

                          precision  recall  f1-score  support
alt.atheism                   0.715   0.589     0.646    319.0
comp.graphics                 0.598   0.614     0.606    389.0
comp.os.ms-windows.misc       0.586   0.551     0.568    394.0
comp.sys.ibm.pc.hardware      0.531   0.620     0.572    392.0
comp.sys.mac.hardware         0.653   0.665     0.659    385.0
comp.windows.x                0.667   0.618     0.641    395.0
misc.forsale                  0.728   0.782     0.754    390.0
rec.autos                     0.789   0.775     0.782    395.0
rec.motorcycles               0.882   0.882     0.882    398.0
rec.sport.baseball            0.703   0.887     0.784    397.0
rec.sport.hockey              0.981   0.810     0.887    827.0
sci.crypt                     0.854   0.841     0.847    396.0
sci.elec

# GS for LR

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
             'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:360: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring().strip(b'\x00').strip()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: to

clf__C:	3
clf__penalty:	l2
clf__solver:	saga
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.816


In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

# GS for RF

In [7]:
from xgboost import XGBClassifier

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror'))])

parameters = {
    'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
    # 'svd__n_components': (250, 500, 750),
    'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__max_iter': (10, 50, 80),
}

gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"RF_1_cv_result_{nb_name}.xlsx")

## Best params

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(
    train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

# param = {'max_depth': 4, 'objective': 'multi:softmax', 'clf__min_child_weight': 1,
#          'booster':'gblinear', # linear model
#          'alpha': 0.0001,  # alpha is the L1 regularizer
#          'lambda': 1} # lambda is the L2 regularizer
# param['num_class'] = len(labels)
# param['eta'] = 0.1 # # scale weight of positive examples
# param['eval_metric'] = ['auc', 'ams@0']


clf = XGBClassifier(objective='multi:softmax', max_depth=4, min_child_weight=1, n_estimators=500, num_class=20, learning_rate=0.075,
                    colsample_bytree=0.7, subsample=0.8, eval_metric='merror')
# clf =  xgb.XGBClassifier(param)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

In [ ]:
# clf.save_model("xgb.bin")
# clf.load_model("xgb.bin")
xgb.plot_importance(clf)
xgb.to_graphviz(clf, num_trees=2)